# climatecasechart, process surya outputs

Merge layout, reading order and ocr into a dataframe.

In [28]:
#@title Imports

import re
import os

import nltk
import json
import string
import openai
import pickle
import pymupdf
import requests
import xlsxwriter
import langdetect
import pandas as pd
from tqdm.cli import tqdm
from multiprocessing import Pool
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.tokenize import TreebankWordTokenizer

# nltk.download('punkt_tab')

tqdm.pandas()

# os.chdir('/slow-data/unitednationsclimatehealth/')
os.chdir('/ipfs-storage/ipfs/herbert/unitednationsclimatehealth/')

keywords = pd.read_excel(
    # 'data/keywords/Physical and Mental Health keywords (30_3_25).xlsx',
    'data/keywords/Physical and Mental Health keywords (2_4_25).xlsx',
    sheet_name='translations'
)
# Fix as per email Marlies 2025-07-14:
# wellbeing missclassified as mental health
keywords.loc[keywords['en'].str.contains('wellbeing'), 'Category'] = 'Physical health'

keywords = keywords.set_index(['Group', 'Category']).copy()

tqdm.pandas()

experiment_date = '2025-03-28-run-01' #@param {"type": "string"}

case_details = pd.read_pickle(f'data/{experiment_date}-case-details.p3')
words = pd.read_pickle(f'data/{experiment_date}-relevant-words.p3')

print(len(case_details), len(words))

350 1432774


In [30]:

keywords[keywords['en'].str.contains('well')]

en         es            de  \
Group             Category                                               
Existing Concepts Physical health  well-being  bienestar  Wohlbefinden   
                  Physical health   wellbeing  bienestar  Wohlbefinden   

                                          fr       nl         it        cs  \
Group             Category                                                   
Existing Concepts Physical health  bien-être  welzijn  benessere    pohodu   
                  Physical health  bien-être  welzijn  benessere  blahobyt   

                                        da      et           fi  ja       no  \
Group             Category                                                     
Existing Concepts Physical health  velvære  heaolu  hyvinvointi  幸福  velvære   
                  Physical health  velvære  heaolu  hyvinvointi  幸福  velvære   

                                                   pl         pt  \
Group             Category                                         
Existing Concepts Physical health  dobre samopoczucie  bem-estar   
                  Physical health  dobre samopoczucie  bem-estar   

                                              sv            uk        ca  \
Group             Category                                                 
Existing Concepts Physical health  välbefinnande  благополуччя  benestar   
                  Physical health  välbefinnande  благополуччя  benestar   

                                       ca.1  
Group             Category                   
Existing Concepts Physical health  benestar  
                  Physical health  benestar

In [31]:
#@title Removed two cases as per online meeting (2025-04-02, 17h, Marlies, David, Kajal, Monique)

# EXCLUDE SLUGS:
#  * 18416
#  * request-for-an-advisory-opinion-on-the-obligations-of-states-with-respect-to-climate-change

excluded_cases = ['18416', 'request-for-an-advisory-opinion-on-the-obligations-of-states-with-respect-to-climate-change']

case_details = case_details[
    ~case_details['slug'].isin(excluded_cases)]

print(len(case_details), len(words))

348 1432774


In [33]:

#@title Remove invalid (empty filename) documents out of 2011-2024 period

case_details['documents'] = case_details['documents'].apply(lambda documents: [
  doc for doc in documents
  if doc['File'].strip() != ''
  and int(doc['Filing Date'][-4:]) in range(2011,2025)
])
case_details = case_details[case_details['documents'].str.len() > 0].copy()
filenames = case_details['documents'].explode().str['File'].str.split('/').str[-1]

words = words[words.index.to_frame()['slug'].isin(case_details['slug'])]
assert words.index.to_frame()['filename'].isin(filenames).min() # shouldn't change

In [34]:
#@title Detect languages

rerun = True #@param {type:"boolean"}

if rerun or 'file_languages' not in globals():
  file_texts = words.groupby(['slug', 'filename'])['text'].apply(' '.join)
  file_languages = file_texts.progress_apply(langdetect.detect)

assert len(set(file_languages) - set(keywords.columns)) == 0

file_languages.rename('language').value_counts().rename('#').reset_index([]).T

100%|██████████| 912/912 [00:19<00:00, 47.03it/s]


language,en,fr,de,es,nl,it,et,no,pl,cs,sv,pt,uk,fi,da
#,603,116,106,22,20,13,6,6,6,4,4,3,1,1,1


In [35]:
#@title Tokenization
#@markdown Takes 15-25', approx 12' more after the second progressbar

rerun = True #@param {type:"boolean"}

tqdm.pandas()

tokenizer = TreebankWordTokenizer().span_tokenize
# ps = PorterStemmer()
# stem = ps.stem

all_upper = re.compile('^[^a-z]+$')
def stem(x):
  """ 'stemming', not really though. Just keeping all-upper texts
  that way, because they are probably abbreviations like AIDS, and
  lowering the rest, to imply case-insensitive for non-abbreviations."""
  return x if all_upper.match(x) else x.lower()

keywords_tokens = keywords.map(
  lambda text: tuple(stem(text[a:b]) for a, b in tokenizer(text)))
# determine max lookahead for the actual articles
max_tokens = keywords_tokens.map(len).max().to_dict()

def tokenize(text):
  return pd.DataFrame([
    {'start': a, 'end': b, 'word': text[a:b], 'token': stem(text[a:b])}
    for a, b in tokenizer(text)
  ])

if rerun or not os.path.exists(f'data/{experiment_date}-tokens.p3'):
  with Pool(20) as pool:
    promises = words['text'].progress_apply(lambda x: pool.apply_async(tokenize, (x, )))
    tokens = promises.progress_apply(lambda x: x.get())

  tokens = pd.concat(list(tokens), keys=tokens.index, names=tokens.index.names)

  tokens['language'] = tokens.groupby(['slug', 'filename'], as_index=False).progress_apply(
    lambda x: pd.Series(file_languages.loc[x.name[0], x.name[1]], index=x.index)).reset_index(0, drop=True)

  tokens.to_pickle(f'data/{experiment_date}-tokens.p3')
else:
  tokens = pd.read_pickle(f'data/{experiment_date}-tokens.p3')

# word-tokens include .,;: etc, e.g. "health,", strip to also match these terms like "health".
tokens['stripped'] = tokens['token'].str.strip('.!?,:;')
if rerun or not os.path.exists(f'data/{experiment_date}-tokens-lookahead.p3'):
  # The token, appended with max_stems (language specific) other tokens, to allow
  # multy-token keyword matching.
  tokens['lookahead'] = tokens.groupby(['slug', 'filename'], as_index=False).progress_apply(
    lambda f: pd.Series(list(zip(*(f['stripped'].shift(-i) for i in range(max_tokens[file_languages[f.name]])))), index=f.index)
  ).reset_index(0, drop=True)
  tokens.to_pickle(f'data/{experiment_date}-tokens-lookahead.p3')
else:
  tokens = pd.read_pickle(f'data/{experiment_date}-tokens-lookahead.p3')

tokens.index.names = list(tokens.index.names[:-1]) + ['token']

100%|██████████| 912/912 [00:10<00:00, 87.22it/s] 


In [36]:
#@title Matching on terms
rerun = True #@param {type:"boolean"}

# allow subsetting on slug and fn orthogonally (or even skip slug alltogether)
assert tokens[[]].reset_index(['slug', 'filename']).groupby(
  'filename')['slug'].apply(set).str.len().max() <= 1

if rerun or 'matches_per_language' not in globals():
  matches_per_language = dict()

for language in keywords_tokens.columns:
  if language in matches_per_language:
    continue

  language_words = tokens[tokens['language'] == language]
  language_tokens = keywords_tokens[language]

  # Filter on the first stems, to speed up the rest with a shortlist
  firsts = set(language_tokens.str[0])
  short_list = language_words['token'].isin(firsts)

  tqdm.pandas(desc=language, leave=False, position=0)

  # extract the terms that match the documents
  matches_per_language[language] = (
    language_words[short_list].progress_apply(
      lambda x: {
        term_tokens for term_tokens in language_tokens
        if x['lookahead'][:len(term_tokens)] == term_tokens
      }, axis=1)
  )
  tqdm.pandas(desc=None, leave=True, position=0)

matches_per_language_ = {k:v[v.str.len() > 0] for k, v in matches_per_language.items() if len(v) > 0 }
matches = pd.concat(list(matches_per_language_.values()))
matches = matches.apply(lambda x: max(x, key=len))
matches


slug                                                                                   filename                       page  position  line  word  token
hugues-falys-fian-greenpeace-ligue-des-droits-humains-v-totalenergies-the-farmer-case  20240301_54998_petition-2.pdf  13    7         0     0     15              (health,)
                                                                                                                            8         0     0     14       (air, pollution)
                                                                                                                      19    9         0     0     11          (well-being,)
                                                                                                                                                  14              (health,)
                                                                                                                      21    6         1     0     10            

In [37]:
#@title Remove overlapping matching, select first longest

#tuple if all integer locations in the token-dataframe for each match
token_ranges = matches.rename('keyword').reset_index().progress_apply(lambda x: set(range(i:=tokens.index.get_loc(tuple(x[:-1])), i+len(x['keyword']))), axis=1)
token_ranges.index = matches.index

#overlaps should be adjacent. Transitive overlaps (e.g. 3 or longer)
# are grouped by the cumsum later on.
overlaps = token_ranges.groupby(['slug', 'filename']).apply(
  lambda file_token_ranges: pd.DataFrame({
  '0': file_token_ranges,
  '1': file_token_ranges.shift(1, fill_value=set()),
  '-1': file_token_ranges.shift(-1, fill_value=set())
}).apply(lambda x: len(x['0'] & x['-1']) + len(x['0'] & x['1']) > 0, axis=1)).reset_index([0, 1], drop=True)

# True each time overlaps is True but was False at the previous match. cumcum then groups these together
overlap = (overlaps & ~overlaps.shift(-1, fill_value=False)).rename('overlap').cumsum()
# The grouping only works if we filter, otherwise the non-overlapping after an overlap would be added too
overlap = overlap[overlaps]

# subset to overlapping matches, process them 1 by 1 by selecting the first longest idx.
overlapping_matches = matches.copy()
overlapping_matches = overlapping_matches[overlaps].reset_index([]).set_index(overlap, append=True)[0]
keep_indices = overlapping_matches.groupby(['slug', 'filename', 'overlap']).apply(lambda x: next(idx[:-1] for idx, m in x.items() if len(m) == max(x.str.len())))

# keep everything without overlap or as selected just now.
keep = ~overlaps | overlaps.index.to_frame().apply(tuple, axis=1).isin(list(map(tuple, keep_indices)))
non_overlapping_matches = matches[keep].copy()

100%|██████████| 12010/12010 [00:03<00:00, 3064.18it/s]


In [38]:
#@title Extract text for each match

words['prefix length'] = words['text'].str.len() + 1 # +1 for ' '.join
words['prefix length'] = (
    words.groupby(
      ['slug', 'filename', 'page', 'position']
    )['prefix length'].cumsum() - words['prefix length'])

translations = keywords.set_index(keywords['en']).apply(lambda x: dict(x.items())).to_dict()

# def document_minorities_window(row):

matches_ = pd.DataFrame({
  k: {
    'start': int(term_words['start'].iloc[0]),
    'end': int(term_words['end'].iloc[-1]),
    # position w.r.t. ' ' joined position (block) instead of word
    'position start': int(first_word['prefix length'] + term_words['start'].iloc[0]),
    'position end': int(last_word['prefix length'] + term_words['end'].iloc[-1]),
    'text': ' '.join(term_words['word']),
    'language': language,
    'translation': translations[language][keyword],
    'paragraph type': first_word['paragraph type'],
    'keyword': keyword,
    'set': keywords[keywords_tokens[file_languages.loc[k[0], k[1]]] == v].index[0][0],
    'type': keywords[keywords_tokens[file_languages.loc[k[0], k[1]]] == v].index[0][1],
  }
  for k, v in tqdm(non_overlapping_matches.items(), total=len(non_overlapping_matches), position=0)
  for term_words in [tokens.iloc[tokens.index.get_loc(k):].iloc[:len(v)]] # alias
  for first_word in [words.loc[k[:-1]]] # alias
  for last_word in [words.loc[term_words.index[-1][:-1]]] # alias
  for language in [file_languages.loc[k[0], k[1]]] # alias
  for keyword in [keywords[keywords_tokens[file_languages.loc[k[0], k[1]]] == v].iloc[0]['en']] # alias
}).T

matches_.index.names = list(tokens.index.names)
matches_

100%|██████████| 11432/11432 [00:16<00:00, 681.99it/s]


start  \
slug                                               filename                      page position line word token         
hugues-falys-fian-greenpeace-ligue-des-droits-h... 20240301_54998_petition-2.pdf 13   7        0    0    15       74   
                                                                                      8        0    0    14       81   
                                                                                 19   9        0    0    11       58   
                                                                                                         14       77   
                                                                                 21   6        1    0    10       48   
...                                                                                                              ...   
anton-foley-and-others-v-sweden-aurora-case        20220510_18245_na.pdf         4    3        2    0    1        16   
                                                                                                         4        39   
                                                                                               3    0    8        52   
                                                                                      8        1    0    3        26   
                                                                                      17       1    0    9        47   

                                                                                                               end  \
slug                                               filename                      page position line word token       
hugues-falys-fian-greenpeace-ligue-des-droits-h... 20240301_54998_petition-2.pdf 13   7        0    0    15     80   
                                                                                      8        0    0    14      9   
                                                                                 19   9        0    0    11     68   
                                                                                                         14     83   
                                                                                 21   6        1    0    10     54   
...                                                                                                             ..   
anton-foley-and-others-v-sweden-aurora-case        20220510_18245_na.pdf         4    3        2    0    1      22   
                                                                                                         4      44   
                                                                                               3    0    8      58   
                                                                                      8        1    0    3      35   
                                                                                      17       1    0    9      50   

                                                                                                               position start  \
slug                                               filename                      page position line word token                  
hugues-falys-fian-greenpeace-ligue-des-droits-h... 20240301_54998_petition-2.pdf 13   7        0    0    15                74   
                                                                                      8        0    0    14                81   
                                                                                 19   9        0    0    11                58   
                                                                                                         14                77   
                                                                                 21   6        1    0    10               136   
...                                                                                                             

In [39]:
#@title Language statistics

n_matches = matches_.groupby(['slug', 'filename'])['keyword'].apply(len)

file_metadata = pd.DataFrame({
  'language': file_languages,
  '# matches': file_languages.index.to_series().apply(n_matches.get).fillna(0).astype(int),
  '# files': 1,
  '# files with matches': file_languages.index.to_series().apply(lambda x: x in n_matches).astype(int),
  'matches': matches_.groupby(['slug', 'filename'])['keyword'].apply(list),
  'texts': matches_.groupby(['slug', 'filename'])['text'].apply(list),
}).sort_values('matches')

language_metadata = file_metadata.groupby('language')[['# files', '# files with matches', '# matches']].sum().astype(int).sort_values('# files', ascending=False)
language_metadata['matches'] = file_metadata.groupby('language')['matches'].apply(lambda x: list({x for x in x.dropna() for x in x}))
language_metadata['texts'] = file_metadata.groupby('language')['texts'].apply(lambda x: list({x for x in x.dropna() for x in x}))
language_metadata

,# files,# files with matches,# matches,matches,texts
language,,,,,
en,603,338,7988,"[leishmaniasis, fearful, emotions, ischaemic h...","[leishmaniasis, health care, fearful, emotions..."
fr,116,86,1693,"[leishmaniasis, emotions, substance abuse, sol...","[anxiété, vaccin, santé, épidémiologie, Zika, ..."
de,106,57,968,"[leishmaniasis, emotions, mortality, malaria, ...","[Bakterien, Gesundheitspflege, Zika, Typhus, B..."
es,22,9,220,"[pandemic, Vector-borne diseases, mood, virus,...","[enfermedades transmitidas por vectores, cript..."
nl,20,18,200,"[morbidity, pandemic, allergens, dehydration, ...","[gele koorts, welzijn, angst, sterfte, bezorgd..."
it,13,7,99,"[bacteria, pandemic, solastalgia, virus, morta...","[zoonosi, tradire, salute, vita umana, virus, ..."
et,6,3,10,"[air pollution, concentrate, well-being]","[heaolu, õhusaaste, keskenduda]"
pl,6,2,5,"[accident, health]","[zdrowie, wypadek]"
no,6,5,25,"[grieved, human life, deaths, loss of life, mo...","[tap av liv, menneskeliv, sørget, dødelighet, ..."


In [40]:
#@title Expand paragraphs and prepare for annotation export

#@markdown with a minimal character window to other paragraph and potentially
#@markdown dilate-cluster them together.

rerun = True #@param {type:"boolean"}
rematch = True #@param {type:"boolean"}
window_size = 250 #@param {"type": "integer"}

if rerun or 'block_texts' not in globals():
  block_texts = words.groupby(['slug', 'filename', 'page', 'position']).agg({
    'text': ' '.join})

if rerun or rematch or 'matched' not in block_texts.columns:
  block_matches = matches_.groupby(['slug', 'filename', 'page', 'position']).agg({
    'text': list, 'keyword': list,
    'position start': list, 'position end': list,
    'set': list, 'type': list,
  }).rename({'text': 'keyword text', 'position start': 'start', 'position end': 'end'}, axis=1)

  block_texts['matched'] = False
  block_texts.loc[block_matches.index, 'matched'] = True
  block_texts['length'] = block_texts['text'].str.len()

  matched_ = block_texts.columns.get_loc('matched')

  for idx in tqdm(block_matches.index, position=0):
    loc = block_texts.index.get_loc(idx)

    # add whole paragraphs to front and back untill at least window_size chars
    prepended, i = 0, 1
    while prepended < window_size and block_texts.index[loc-i][:2] == idx[:2]:
      prepended += block_texts.iloc[loc-i]['length']
      block_texts.iloc[loc-i, matched_] = True
      i += 1

    appended, i = 0, 1
    while appended < window_size and block_texts.index[loc+i][:2] == idx[:2]:
      appended += block_texts.iloc[loc+i]['length']
      block_texts.iloc[loc+i, matched_] = True
      i += 1

  block_texts['block'] = block_texts.groupby(['slug', 'filename'], as_index=False).apply(
    lambda x: x['matched'] & ~x['matched'].shift(1, fill_value=False)).cumsum().reset_index(0, drop=True)

matched_block_texts = block_texts.query('matched').copy()

# offset character ranges to blocks
matched_block_texts['block offset'] = matched_block_texts['text'].str.len() + 2
matched_block_texts['block offset'] = matched_block_texts.groupby(['slug', 'filename', 'block'], as_index=False)['block offset'].cumsum() - matched_block_texts['block offset']

matched_block_texts[block_matches.columns] = block_matches
matched_block_texts[block_matches.columns] = matched_block_texts[block_matches.columns].map(lambda x: x if x==x else [])
matched_block_texts['start'] = matched_block_texts.apply(lambda x: [v + x['block offset'] for v in x['start']], axis=1)
matched_block_texts['end'] = matched_block_texts.apply(lambda x: [v + x['block offset'] for v in x['end']], axis=1)

# Group blocks to 1 row each
grouped_blocks = matched_block_texts.reset_index(['page']).groupby(['slug', 'filename', 'block']).agg({
  'page': set, 'text': '\n\n'.join,
  'keyword text': lambda x: [x for x in x for x in x],
  'keyword': lambda x: [x for x in x for x in x],
  'start': lambda x: [x for x in x for x in x],
  'end': lambda x: [x for x in x for x in x],
  'set': lambda x: [x for x in x for x in x],
  'type': lambda x: [x for x in x for x in x],
})

grouped_blocks_export = grouped_blocks.copy()
grouped_blocks_export['page'] = grouped_blocks_export['page'].apply(lambda x: ', '.join(map(str, x)))
grouped_blocks_export = grouped_blocks_export.set_index(['page'], append=True)[['keyword', 'keyword text', 'text', 'start', 'end', 'set', 'type']].swaplevel('page', 'block', axis=0).sort_index()
grouped_blocks.to_pickle(f'data/{experiment_date}-match-blocks.p3')
grouped_blocks_export

100%|██████████| 6713/6713 [00:04<00:00, 1677.93it/s]


keyword  \
slug                                               filename                           page   block                                                      
11477                                              20231011_11477_decision.pdf        1      1                                        [air pollution]   
                                                                                             2                              [concentrate, well-being]   
                                                                                             3      [air pollution, air pollution, air pollution, ...   
4m2-action-during-covid-19-pandemic-in-geneva-s... 20220713_75397_decision.pdf        12     7                                               [health]   
                                                                                      13     8                                             [pandemic]   
...                                                                                                                                               ...   
youth-for-climate-justice-v-austria-et-al          20240409_3937120_press-release.pdf 2      3818                 [well-being, mental health, health]   
                                                                                      3      3819                                        [well-being]   
zad-de-la-colline-holcim-switzerland               20220124_75405_decision.pdf        20, 21 3820                                [health, well-being]   
                                                   20220629_75405_decision.pdf        19, 20 3821                            [health, health, health]   
                                                                                      21, 22 3822                                            [health]   

                                                                                                                                         keyword text  \
slug                                               filename                           page   block                                                      
11477                                              20231011_11477_decision.pdf        1      1                                            [õhusaaste]   
                                                                                             2                                   [keskenduda, heaolu]   
                                                                                             3      [õhusaaste, õhusaaste, õhusaaste, õhusaaste, õ...   
4m2-action-during-covid-19-pandemic-in-geneva-s... 20220713_75397_decision.pdf        12     7                                                [santé]   
                                                                                      13     8                                             [pandémie]   
...                                                                                                                                               ...   
youth-for-climate-justice-v-austria-et-al          20240409_3937120_press-release.pdf 2      3818                 [well-being, mental health, health]   
                                                                                      3      3819                                        [well-being]   
zad-de-la-colline-holcim-switzerland               20220124_75405_decision.pdf        20, 21 3820                                  [santé, bien-être]   
                                                   20220629_75405_decision.pdf        19, 20 3821                               [santé, santé, santé]   
                                                                                      21, 22 3822                                             [santé]   

                                                                                                                                                 text  \
slug     

In [41]:
#@title Export ranked

# ranked w.r.t. keywords and how often they occurrence in matches, documents
# contained in the highest ranked sheet only if matched on multiple keywords.

rerun = True #@param {type:"boolean"}
destination = 'data/2025-04-02-except-exports/' #@param {"type": "string"}

!echo scp -O -r il-c03:`pwd`/'2025-04-02-keyword-rank-sample.xlsx' .

def write_sheet(blocks, worksheet):
  bold = workbook.add_format({"bold": True, 'valign': 'top', 'align': 'left', "text_wrap": True})
  kw_bold = workbook.add_format({"bold": True, 'valign': 'bottom', 'align': 'left', "text_wrap": True})
  kw_bold = workbook.add_format({"bold": True, 'valign': 'bottom', 'align': 'left', "text_wrap": True})

  highlighted_p = workbook.add_format({
      "bold": True, 'valign': 'top', 'align': 'left', "text_wrap": True,
      'font_color': '009CEF'})
  highlighted_m = workbook.add_format({
      "bold": True, 'valign': 'top', 'align': 'left', "text_wrap": True,
      'font_color': 'DC002D'})
  highlighted_g = workbook.add_format({
      "bold": True, 'valign': 'top', 'align': 'left', "text_wrap": True,
      'font_color': '60B669'})
  formats = {'Physical health': highlighted_p, 'Mental health': highlighted_m, 'General Health': highlighted_g}
  cell = workbook.add_format({"text_wrap": True, 'valign': 'top', 'align': 'left'})

  def xlsx_highlight(sample, formats):
    done = 0
    text = sample['text']
    items = []
    for a, b, t in zip(sample['start'], sample['end'], sample['type']):
      before = text[done:a]
      highlight = text[a:b]
      if before != '':
        items.extend([before])
      if highlight != '':
        items.extend([formats[t], highlight])
      done = b
    if text[done:] != '':
      items.append(text[done:])
    return items

  slug_column = 0
  fn_column = 1
  pages_column = 2
  text_column = 3
  terms_column = 4
  hidden_text_column = 15

  worksheet.write_string(0, slug_column, "link", bold)
  worksheet.set_column(slug_column, slug_column, 4)
  worksheet.write_string(0, fn_column, "filename", bold)
  worksheet.set_column(fn_column, fn_column, 20)
  worksheet.write_string(0, pages_column, "pages", bold)
  worksheet.set_column(pages_column, pages_column, 5)
  worksheet.write_string(0, terms_column, "keyword", bold)
  worksheet.set_column(terms_column, terms_column, 20)
  worksheet.write_string(0, text_column, "text", bold)
  worksheet.set_column(text_column, text_column, 150)

  worksheet.write_string(0, text_column, "hidden text", bold)
  worksheet.set_column(hidden_text_column, hidden_text_column, 150)

  keywords = blocks['keyword text'].explode()

  row_num=1

  for slug, slug_keywords in keywords.groupby('slug'):
    if len(slug_keywords) > 1:
      worksheet.merge_range(row_num, slug_column, row_num + len(slug_keywords)-1, slug_column, "")
    worksheet.write_string(row_num, slug_column, slug, bold)
    for fn, file_keywords in slug_keywords.groupby('filename'):
      if len(file_keywords) > 1:
        worksheet.merge_range(row_num, fn_column, row_num + len(file_keywords)-1, fn_column, "")
      worksheet.write_string(row_num, fn_column, fn, bold)
      for pages, page_keywords in file_keywords.groupby('page'):
        if len(page_keywords) > 1:
          worksheet.merge_range(row_num, pages_column, row_num + len(page_keywords)-1, pages_column, "")
        worksheet.write_string(row_num, pages_column, pages, bold)
        for block, block_keywords in page_keywords.groupby('block'):
          row = blocks.loc[slug, fn, pages, block]
          terms = row['keyword text']
          if len(block_keywords) > 1:
            worksheet.merge_range(row_num, text_column, row_num + len(block_keywords)-1, text_column, "")
          worksheet.write_rich_string(row_num, text_column, *xlsx_highlight(row, formats), cell)
          worksheet.write_rich_string(row_num, hidden_text_column, *xlsx_highlight(row, formats), cell)

          for _, keyword in block_keywords.items():
            worksheet.write_string(row_num, terms_column, keyword, kw_bold)
            row_num+=1


cnt = matches_['keyword'].value_counts()
keyword_ranking = cnt.reset_index()['keyword'].reset_index().set_index('keyword')['index'].rename('rank')
filename_rank = matches_.groupby(['slug', 'filename'])['keyword'].apply(lambda x: keyword_ranking.loc[x].min()).sort_values().rename('rank')
rank_cnt = filename_rank.value_counts()

filename_samples = filename_rank.reset_index().groupby('rank').apply(
  lambda x: x.sample(rank_cnt[x.name])[['slug', 'filename']], include_groups=False)

workbook = xlsxwriter.Workbook(f"2025-04-02-keyword-rank-sample.xlsx")

worksheet = workbook.add_worksheet(f'all')
block = grouped_blocks_export[
    grouped_blocks_export['keyword text'].str.len() > 0]
write_sheet(block, worksheet)

for rank, filenames in filename_samples.groupby('rank')['filename']:
  keyword = keyword_ranking[keyword_ranking==rank].index[0]
  worksheet = workbook.add_worksheet(f'{keyword} ({cnt[keyword]})')
  block = grouped_blocks_export[
      grouped_blocks_export.index.to_frame()['filename'].isin(filenames)]
  block = block[block['keyword text'].str.len() > 0]
  write_sheet(block, worksheet)
workbook.close()

!echo scp -O -r il-c03:`pwd`/'2025-04-02-keyword-rank-sample.xlsx' .

scp -O -r il-c03:/ipfs-storage/ipfs/herbert/unitednationsclimatehealth/2025-04-02-keyword-rank-sample.xlsx .
scp -O -r il-c03:/ipfs-storage/ipfs/herbert/unitednationsclimatehealth/2025-04-02-keyword-rank-sample.xlsx .


In [42]:
#@title Export keyword list with tons of metadata regarding also case and document
matches__ = matches_.rename({
  'language': 'Document Language', 'set': 'Keyword Group', 'type': 'Keyword Type',
  'keyword': 'Keyword', 'text': 'Keyword Text', 'translation': 'Keyword Translation',
  'paragraph type': 'Paragraph Type'}, axis=1)
matches__ = matches__[[
  'Keyword Text', 'Document Language', 'Keyword Translation', 'Paragraph Type',
  'Keyword', 'Keyword Group', 'Keyword Type']]

documents = case_details.set_index('slug')['documents'].explode().apply(pd.Series).rename({
  'Type': 'Document Type', 'File': 'Document Link', 'Filing Date': 'Document Filing Date',
  'Summary': 'Document Summary'
}, axis=1)
documents['filename'] = documents['Document Link'].str.split('/').str[-1]
documents = documents.set_index('filename', append=True)
documents['Document Filing Year'] = documents['Document Filing Date'].str[-4:].astype(int)
documents = documents[['Document Filing Year', 'Document Filing Date', 'Document Type', 'Document Link', 'Document Summary']]

cases = cases = case_details.groupby('slug').agg(dict(
  **{'country': '|'.join, 'Reporter Info': lambda x: '|'.join(v if v==v else '' for v in x)},
  **{c: 'first' for c in [
      'description', 'link', 'slug', 'title', 'non-english title',
      'Filing Date', 'Status', 'Summary', 'At Issue']})).rename({
  'title': 'Case Title', 'non-english title': 'Non-english Case Title', 'Filing Date': 'Case Filing Date', 'Status': 'Case Status',
  'country': 'Case Country', 'link': 'Case Link', 'description': 'Case Description'
}, axis=1)
cases_ = dict(cases.iterrows())
metadata = pd.concat([
  documents.index.to_frame()['slug'].apply(cases_.get), documents
], axis=1)
metadata_ = dict(metadata.iterrows())

metadata__ = matches__.index.to_frame()[['slug', 'filename']].apply(lambda x: metadata_.get(tuple(x)), axis=1)
aggregation_export = pd.concat([metadata__, matches__], axis=1).dropna(subset = ['Case Title']).reset_index(['position', 'line', 'word', 'token'], drop=True)
aggregation_export.to_excel(f'data/2025-04-02-matches.xlsx')
aggregation_export.to_pickle(f'data/2025-04-02-matches.p3')
aggregation_export.drop(['slug'], axis=1).reset_index().to_excel(f'data/2025-04-02-matches-flat.xlsx', index=False)
!echo scp -O il-c03:`pwd`'/data/2025-04-02-matches{,-flat}.xlsx' .

scp -O il-c03:/ipfs-storage/ipfs/herbert/unitednationsclimatehealth/data/2025-04-02-matches{,-flat}.xlsx .


In [43]:
#@title Export statistics requested by Marlies Hesselman (email, 2025-04-01)

years = case_details.groupby('slug')['Filing Date'].first().to_dict()
matches_['year'] = matches_.index.to_frame()['slug'].apply(years.get).dropna().astype(int)
m = matches_.query('year >= 2011').query('year <= 2024')

yearly = m.groupby(['set', 'year', 'type']).apply(len, include_groups=False).reset_index().pivot(index=['set', 'year'], columns=['type'], values=0).fillna(0).astype(int)
yearly['Total'] = yearly.sum(1).rename('Total')
yearly = pd.concat([yearly, pd.DataFrame({('Total', ''): yearly.sum(0)}).T], axis=0)

overview = m.groupby(['set', 'type']).apply(len, include_groups=False).reset_index().pivot(index='set', columns='type', values=0).fillna(0).astype(int)
overview['Total'] = overview.sum(1).rename('Total')
overview.loc['Total'] = overview.sum(0).rename('Total')

per_keyword = m.groupby(['set', 'keyword', 'type', 'year']).apply(len, include_groups=False).reset_index().pivot(index=['set', 'type', 'keyword'], columns='year', values=0).fillna(0).astype(int)
per_keyword.columns = list(map(int, per_keyword.columns))
a = set(keywords['en'].rename('keyword').reset_index().apply(tuple, axis=1))
b = set(per_keyword.index.to_frame().apply(tuple, axis=1))
b = pd.DataFrame(list(a-b), columns=['set', 'type', 'keyword']).set_index(['set', 'type', 'keyword'])
b[per_keyword.columns] = 0
per_keyword = pd.concat([per_keyword, b], axis=0).sort_index()
per_keyword['Total'] = per_keyword.sum(1).rename('Total')
per_keyword.index.names = ['set', 'type', 'keyword']
per_keyword = pd.concat([per_keyword, pd.DataFrame({('Total', '', ''): per_keyword.sum(0)}).T], axis=0)

per_keyword.index.names = ['set', 'type', 'keyword']
per_keyword.reset_index().sort_values(['set', 'type', 'Total'], ascending=False).set_index(['set', 'type', 'keyword'])

# mental/phsyical/total docums / cases
ucases = matches_.reset_index().groupby('type')['slug'].nunique()
ufiles = matches_.reset_index().groupby('type')['filename'].nunique()
ucases['Total'] = len(case_details['slug'].unique())
ufiles['Total'] = case_details['documents'].str.len().sum()
type_stats = pd.DataFrame({
  '# cases': ucases,
  '# files': ufiles,
})

with pd.ExcelWriter(f"data/2025-04-02-keyword-stats.xlsx") as f:
  type_stats.to_excel(f, sheet_name='keyword types')
  overview.to_excel(f, sheet_name='overview')
  yearly.to_excel(f, sheet_name='yearly')
  per_keyword.to_excel(f, sheet_name='keywords')
!echo scp -O il-c03:`pwd`'/data/2025-04-02-keyword-stats.xlsx' .

scp -O il-c03:/ipfs-storage/ipfs/herbert/unitednationsclimatehealth/data/2025-04-02-keyword-stats.xlsx .


In [ ]:
#@title OCR 'typo' estimates

# hacky way to get an idea about OCR errors, particularly ocr fails for cursive/italic text.
# they are probably usually

from nltk.corpus import words as english, brown
vocab = {x.lower() for x in set(brown.words()) | set(english.words())} | {
  'co2', "'s", 'http',
}
en_tokens = tokens.query('language == "en"')

# if the token is all weird symbols, don't spell check it. Usually numbers, urls, etc.
drop = en_tokens['token'].str.contains('^(?:[0-9.,\\/%\'℃":"@\(\)\[\]\{\}#!^&"• 5 1 … . \| \<°·\-\>§“$2”=–¶©4]|°C|id\.)*$')
en_tokens = en_tokens[~drop]
drop = en_tokens['token'].str.contains('^[A-Z\.]{2,}$')
en_tokens = en_tokens[~drop]

known_word = en_tokens['token'].str.lower().str.strip(' .!').isin(vocab)
q = en_tokens[~known_word]['token'].value_counts()
a = words.loc[known_word[known_word].index]['paragraph type'].value_counts()
b = words.loc[known_word[~known_word].index]['paragraph type'].value_counts()

# check spelling.
pd.DataFrame({
  'errors (%)': (100 * b / (a + b)).round(2).sort_values(ascending=False),
  'words': words['paragraph type'].value_counts()
}).sort_values('errors (%)')

,errors (%),words
paragraph type,,
ListItem,4.76,370710
Text,4.96,611226
Form,8.87,2279
TableOfContents,9.53,15789
SectionHeader,9.92,41920
TextInlineMath,10.16,3048
Code,10.99,40
Table,13.13,76981
Caption,15.01,1459
